# Identifying Profitable App Categories from the App Store and Google Play Store

Our goal is to isolate app profitable app categories in the App Store and Google Play Store. "Profitability", for the purposes of our project, will loosely correlate to number of users. This is driven by a desire to create apps whose primary revenue stream comes from ad revenue. Once profitable app categories are identified, we will be able to begin predesign on our own app(s) in an effort to maximlize future profit.

As of September 2018, there were approximately 2 million App Store apps, and 2.1 million Google Play Store apps. Instead of analyzing all apps in totality, we will focus on a small subset of apps. Specifically, we will analyze 10,000 Android apps available on the Google Play Store (this dataset was collected in August 2018) and approximately 7,000 iOS apps available on the App Store (collected in July 2017).

The below function will provide us with an easy way to explore our datasets. It takes 4 parameters. `dataset` takes in a 
dataset. `start` and `end` represent indices of the dataset. `rows_and_columns` is a Boolean value; when True, it the number 
of rows and columns will be printed along with the output.



In [1]:
def explore_data(dataset, start, end, rows_and_columns=False):
    dataset_slice = dataset[start:end]    
    for row in dataset_slice:
        print(row)
        print('\n') # adds a new (empty) line after each row

    if rows_and_columns:
        print('Number of rows:', len(dataset))
        print('Number of columns:', len(dataset[0]))

In [2]:
from csv import reader

open_file = open("AppleStore.csv", encoding='utf8')
reader_file = reader(open_file)

appstore = list(reader_file)
appstore_header = appstore[0]
appstore = appstore[1:]

open_file = open("googleplaystore.csv", encoding='utf8')
reader_file = reader(open_file)

playstore = list(reader_file)
playstore_header = playstore[0]
playstore = playstore[1:]

print(appstore_header)
explore_data(appstore, 0, 4, rows_and_columns = True)

print(playstore_header)
explore_data(playstore, 0, 4, rows_and_columns = True)


['id', 'track_name', 'size_bytes', 'currency', 'price', 'rating_count_tot', 'rating_count_ver', 'user_rating', 'user_rating_ver', 'ver', 'cont_rating', 'prime_genre', 'sup_devices.num', 'ipadSc_urls.num', 'lang.num', 'vpp_lic']
['284882215', 'Facebook', '389879808', 'USD', '0.0', '2974676', '212', '3.5', '3.5', '95.0', '4+', 'Social Networking', '37', '1', '29', '1']


['389801252', 'Instagram', '113954816', 'USD', '0.0', '2161558', '1289', '4.5', '4.0', '10.23', '12+', 'Photo & Video', '37', '0', '29', '1']


['529479190', 'Clash of Clans', '116476928', 'USD', '0.0', '2130805', '579', '4.5', '4.5', '9.24.12', '9+', 'Games', '38', '5', '18', '1']


['420009108', 'Temple Run', '65921024', 'USD', '0.0', '1724546', '3842', '4.5', '4.0', '1.6.2', '9+', 'Games', '40', '5', '1', '1']


Number of rows: 7197
Number of columns: 16
['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver']
['Photo Editor

Above, we can see that there are 7,197 apps and 16 columns in our App Store dataset. Columns that appear to be relevant for our purposes: `track_name`, `rating_count_tot`, `rating_count_ver`, and `prime_genre`. Similarly, we can see that there are 10,841 apps and 13 columns in our Play Store dataset. Columns that appear to be relevant for our purposes: `App`, `Category`, `Rating`, `Reviews`, `Installs`, `Type`, and `Genres`.

As stated previously, we will focus on apps that can provide a desirable revenue stream from ad revenue. We will therefore focus on free apps. Relatedly, our app will be developed for a primarily English audience. Our analaysis will therefore focus on free, English apps. Before we can begin our analysis in earnest, we will first need to clean our datasets.

In [3]:
# Row 10472 in our Google Play Store dataset has a missing category. We will opt to remove this datapoint.
del playstore[10472]

From previous research, we know that the Google Play Store dataset contains duplicate entries for some apps. We will need to remove these duplicate entries prior to proceeding. Let's first confirm that we do in fact have duplicates, and take a look at some examples.

In [4]:
duplicate_apps = []
unique_apps = []

for app in playstore:
    appname = app[0]
    
    if appname in unique_apps:
        duplicate_apps.append(appname)
    else:
        unique_apps.append(appname)
        
print("Number of duplicate apps: " + str(len(duplicate_apps)))
print('\n')
print('Examples of duplicate apps:', duplicate_apps[:7])
print('\n')  

print("Number of unqiue apps: " + str(len(unique_apps)))

Number of duplicate apps: 1181


Examples of duplicate apps: ['Quick PDF Scanner + OCR FREE', 'Box', 'Google My Business', 'ZOOM Cloud Meetings', 'join.me - Simple Meetings', 'Box', 'Zenefits']


Number of unqiue apps: 9659


We will now take a closer look at one of the apps that we know has duplicates--Box.

In [5]:
for app in playstore:
    if app[0] == 'Quick PDF Scanner + OCR FREE':
        print(app)

['Quick PDF Scanner + OCR FREE', 'BUSINESS', '4.2', '80805', 'Varies with device', '5,000,000+', 'Free', '0', 'Everyone', 'Business', 'February 26, 2018', 'Varies with device', '4.0.3 and up']
['Quick PDF Scanner + OCR FREE', 'BUSINESS', '4.2', '80805', 'Varies with device', '5,000,000+', 'Free', '0', 'Everyone', 'Business', 'February 26, 2018', 'Varies with device', '4.0.3 and up']
['Quick PDF Scanner + OCR FREE', 'BUSINESS', '4.2', '80804', 'Varies with device', '5,000,000+', 'Free', '0', 'Everyone', 'Business', 'February 26, 2018', 'Varies with device', '4.0.3 and up']


From this example, we can see that the rows are essentially the same, with the exception of variance in the 4th column in one row. This corresponds to the number of reviews. Essentially, what is happening here is that the number of reviews was tracked on different occasions, and in some cases, instead if simply being updated in line, a new row was created. We also have some truly duplicate rows, where all information is the same, such as with the other two rows that both display the same number of reviews.

To ensure we have the most up to date data, we will assume that in cases with duplicates, we will always keep the row that contains the most reviews, so the one with the highest integer value in the 4th column.

To accomplish our goal, we will:
- Create a dictionary that contains each app, and the highest number of reviews for that app
- Build a new dataset that only includes a single entry for each app, using results from the dictionary.

In [6]:
reviews_max = {}

for app in playstore:
    appname = app[0]
    num_reviews = float(app[3])
    
    if appname in reviews_max and reviews_max[appname] < num_reviews:
        reviews_max[appname] = num_reviews
    elif appname not in reviews_max:
        reviews_max[appname] = num_reviews


reviews_max["Quick PDF Scanner + OCR FREE"]

80805.0

If our dictionary has been created correctly, we should have 9659 key-value pairs, which matches the amount in our unique app test above. Let's check that.

In [7]:
len(reviews_max)

9659

Now that our dictionary has been created, we will create a new dataset including only unique entries. We will do this by:
- Creating a new list, `playstore_cleaned`, which will store our cleaned dataset
- Creating a new list, `already_added`, which will enable us to exclude already added apps as we create the new dataset
- Iterate through our original `playstore` dataset:
    - If an app has not already been added and if the number of reviews for a given row matches the corresponding value in the `reviews_max` dictionary, add the app to the `playstore_cleaned` dataset and add the name to the `already_added` list
    - If the app already exists in the `already_added` list, skip over it and move on to the next app

In [16]:
playstore_cleaned = []
already_added = []

for app in playstore:
    name = app[0]
    num_reviews = float(app[3])
    
    if reviews_max[name] == num_reviews and name not in already_added:
        playstore_cleaned.append(app)
        already_added.append(name)
print("Total apps in cleaned list", len(playstore_cleaned))   
print('\n')
print("Sample apps:")
print('\n')
explore_data(playstore_cleaned, 0, 7)


Total apps in cleaned list 9659


Sample apps:


['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up']


['Sketch - Draw & Paint', 'ART_AND_DESIGN', '4.5', '215644', '25M', '50,000,000+', 'Free', '0', 'Teen', 'Art & Design', 'June 8, 2018', 'Varies with device', '4.2 and up']


['Pixel Draw - Number Art Coloring Book', 'ART_AND_DESIGN', '4.3', '967', '2.8M', '100,000+', 'Free', '0', 'Everyone', 'Art & Design;Creativity', 'June 20, 2018', '1.1', '4.4 and up']


['Paper flowers instructions', 'ART_AND_DESIGN', '4.4', '167', '5.6M', '50,000+', 'Free', '0', 'Everyone', 'Art & Design', 'March 26, 2017', '1.0', '2.3 and up']


['Smoke Effect Photo Maker - Smoke Editor'

The App Store dataset does not contain any duplicates. It uses an `id` column, and each entry is unique. However, both datasets do contain non-English apps, so we will now need to remove those, per our previously stated goal. Every character corresponds to an ASCII value. Characters commonly used in the English language make up the range 0 to 127. Therefore, we should be able to identify and remove non-English apps by looking for those apps whose name includes an ASCII value of 128 or greater.

To facilitate the identification of English apps, we will start by writing a function that can determine if an app is English or not.

In [40]:
def is_english(appname):
    for letter in appname:
        if ord(letter) > 127:
            return False
        
    return True

In [56]:
print(is_english("لعبة تقدر تربح DZ"))
print(is_english("中国語 AQリスニング"))
print(is_english("Easy Realistic Drawing Tutorial"))


False
False
True


This looks good; however, we may encounter apps that fail this check even if they truly are English apps. For example, if we have an app with an emoji (i.e. 😜) in the title, our function may determine it is not English. This is because these sorts of characters fall out of the ASCII range of 0 - 127

In [57]:
print(ord("😜"))

128540


To mitigate data loss as much as possible, we will update our function to only exclude apps where at least three characters in the app name exceed an ASCII value of 127.

In [61]:
def is_english(appname):
    non_english_letters = 0
    for letter in appname:
        if ord(letter) > 127:
            non_english_letters += 1
        if non_english_letters >= 3:    
            return False
        
    return True

In [67]:
print(is_english('Instachat 😜'))
print(is_english('Docs To Go™ Free Office Suite'))
print(is_english('爱奇艺PPS -《欢乐颂2》电视剧热播'))

True
True
False


Now that our function has been updated, let's create new lists only containing English apps, based on the findings of our `is_english` function.

In [75]:
playstore_english = []
appstore_english = []

for app in playstore_cleaned:
    if is_english(app[0]) == True:
        playstore_english.append(app)

for app in appstore:
    if is_english(app[1]) == True:
        appstore_english.append(app)

Finally, we need to remove apps that are not free, since we want to only focus on free apps for our analysis. We will create our 'final' datasets below:

In [81]:
playstore_final = []
appstore_final = []

for app in playstore_english:
    price = app[7]
    
    if price == '0':
        playstore_final.append(app)

for app in appstore_english:
    price = app[4]
    
    if price == '0.0':
        appstore_final.append(app)


Before we begin our analysis, let's take a look at how many apps we are dealing with in each respective dataset.

In [88]:
print("Our final Google Play store dataset contains", len(playstore_final), " apps.")
print("Our final Apple Store dataset contains", len(appstore_final), " apps.")

Our final Google Play store dataset contains 8848  apps.
Our final Apple Store dataset contains 3203  apps.


Our ultimate goal for app development is to first develop an app for Android, measure its success, and if it is ultimately profitable after six months, develop an iOS version of the app. As such, we want to make sure we focus on apps that are successful in both markets, as modifying our future app(s) off of those should increase overall profitability.

To start with, we will want to compare genres of apps; specifically, we want to see what genres of apps are profitable in both the Android and iOS marketplaces.

In [92]:
print(playstore_header)
print(playstore_final[27])
print("\n")
print(appstore_header)
print(appstore_final[27])

['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver']
['Easy Realistic Drawing Tutorial', 'ART_AND_DESIGN', '4.1', '223', '4.2M', '100,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 22, 2017', '1.0', '2.3 and up']


['id', 'track_name', 'size_bytes', 'currency', 'price', 'rating_count_tot', 'rating_count_ver', 'user_rating', 'user_rating_ver', 'ver', 'cont_rating', 'prime_genre', 'sup_devices.num', 'ipadSc_urls.num', 'lang.num', 'vpp_lic']
['543186831', '8 Ball Pool™', '86776832', 'USD', '0.0', '416736', '19076', '4.5', '4.5', '3.9.1', '4+', 'Games', '38', '5', '10', '1']


Below, we will create two different freuency tables. One for the `prime_genre` column in the Apple Store dataset, and one for the `Genres` column in the Google Play Store dataset. We will then convert that frequency table to display percentages (when compared to the totality of apps in the respective app store) and sort by percentage in descending order.

In [110]:
def freq_table(dataset, index):
    frequencies = {}
    
    for app in dataset:
        if app[index] in frequencies:
            frequencies[app[index]] += 1
        else:
            frequencies[app[index]] = 1
    
    for k in frequencies:
        frequencies[k] = frequencies[k] / len(dataset)
    return frequencies

    

In [114]:
def display_table(dataset, index):
    table = freq_table(dataset, index)
    table_display = []
    for key in table:
        key_val_as_tuple = (table[key], key)
        table_display.append(key_val_as_tuple)

    table_sorted = sorted(table_display, reverse = True)
    for entry in table_sorted:
        print(entry[1], ':', entry[0])

In [115]:
display_table(appstore_final, 11)

Games : 0.5825788323446769
Entertainment : 0.07836403371838901
Photo & Video : 0.049953168904152356
Education : 0.036840462066812366
Social Networking : 0.033093974399000935
Shopping : 0.025913206369029034
Utilities : 0.024664377146425227
Sports : 0.021542304089915705
Music : 0.020605682172962846
Health & Fitness : 0.020293474867311895
Productivity : 0.017483609116453323
Lifestyle : 0.015610365282547611
News : 0.013424914142990947
Travel : 0.012488292226038089
Finance : 0.010927255697783328
Weather : 0.008741804558226662
Food & Drink : 0.008117389946924758
Reference : 0.005307524196066188
Business : 0.005307524196066188
Book : 0.003746487667811427
Navigation : 0.0018732438339057135
Medical : 0.0018732438339057135
Catalogs : 0.001248829222603809


In the Apple Store, Games make up a whopping 58.3% of total apps (again, remember this is limited in scope to free English apps, so we must be careful not to generalize outside of this analyzed sample). The second highest category is Entertainment apps, at 7.8%. Between those two categories, which can both broadly be categorized as entertainment, over 66%, or 2/3rds of all apps in our sample, are entertainment apps, and that doesn't even incldue other cateogires such as Sports and Music, which may also contain a lot of apps focused on entertainment rather than utility.

Out of the categories that are clearly focused on utility, we have:
- Photo & Video: 5% 
- Education: 3.7%
- Productivity: 1.7%

Those make up just over 10%, which leaves other various categories to account for roughly 24% of apps. Based on the above, it seems clear that entertainment apps, and games in particular, make up the vast majority of Apple Store apps. That being said, this isn't itself evidence that those apps are the most widely used. Let's keep this knowledge off to the side and revisit later.

In [119]:
display_table(playstore_final, 9)

Tools : 0.08442585895117541
Entertainment : 0.06080470162748644
Education : 0.05357142857142857
Business : 0.04599909584086799
Productivity : 0.03899186256781193
Lifestyle : 0.038765822784810125
Finance : 0.037070524412296565
Medical : 0.035375226039783
Sports : 0.03458408679927667
Personalization : 0.03322784810126582
Communication : 0.032323688969258586
Action : 0.03096745027124774
Health & Fitness : 0.030854430379746837
Photography : 0.029498191681735987
News & Magazines : 0.02802893309222423
Social : 0.02667269439421338
Travel & Local : 0.02328209764918626
Shopping : 0.02249095840867993
Books & Reference : 0.021360759493670885
Simulation : 0.020456600361663652
Dating : 0.018648282097649186
Arcade : 0.01842224231464738
Video Players & Editors : 0.017744122965641953
Casual : 0.01763110307414105
Maps & Navigation : 0.013901446654611212
Food & Drink : 0.012432188065099457
Puzzle : 0.011301989150090416
Racing : 0.009945750452079566
Role Playing : 0.009380650994575045
Libraries & Demo : 

From looking at the Google Play dataset, we can see that the genres are a lot more broad. It also appears that apps on the Google Play store are more varied in general. Given that we know that games and entertainment apps constitute 2/3rds of Apple Store apps, let's add up the precentage of these comparables categories in the Google Play dataset.

- Entertainment: 0.06
- Action: 0.03
- Simulation: 0.02
- Arcade: 0.02
- Casual: 0.02
- Puzzle: 0.01
- Racing: 0.01
- Role Playing: 0.01
- Adventure: 0.01
- Card: 0.005
- Trivia: 0.004
- Casino: 0.004
- Board: 0.004

There are many other related categories below this, but the percentage gets very small very quickly. But with the above apps, we can see that over 20% of the apps are games and entertainment. This is a much smaller percentage than what we have in the Apple Store dataset, but it's stil lthe largest category in respect to the rest of the dataset. 

However, games make up a much smaller proportion of entertainment apps in the Google Play store than they do in the Apple Store, whereas apps more broadly categorized as 'entertainment' are comparable: they make up 7.8% of the Apple Store dataset and 6% of the Google Play Store dataset. 

In [129]:
print(playstore_header)
print(appstore_header)

['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver']
['id', 'track_name', 'size_bytes', 'currency', 'price', 'rating_count_tot', 'rating_count_ver', 'user_rating', 'user_rating_ver', 'ver', 'cont_rating', 'prime_genre', 'sup_devices.num', 'ipadSc_urls.num', 'lang.num', 'vpp_lic']


In [134]:
genres_appstore = freq_table(appstore_final, 11)

for genre in genres_appstore:
    total = 0
    len_genre = 0
    for app in appstore_final:
        app_genre = app[11]
        if app_genre == genre:
            num_ratings = float(app[5])
            total += num_ratings
            len_genre += 1
    avg_num_ratings = total / len_genre
    print(genre, ':', avg_num_ratings)
        

Social Networking : 71548.34905660378
Photo & Video : 28441.54375
Games : 22886.36709539121
Music : 57326.530303030304
Reference : 79350.4705882353
Health & Fitness : 23298.015384615384
Weather : 52279.892857142855
Utilities : 19156.493670886077
Travel : 28243.8
Shopping : 27230.734939759037
News : 21248.023255813954
Navigation : 86090.33333333333
Lifestyle : 16815.48
Entertainment : 14195.358565737051
Food & Drink : 33333.92307692308
Sports : 23008.898550724636
Book : 46384.916666666664
Finance : 32367.02857142857
Education : 7003.983050847458
Productivity : 21028.410714285714
Business : 7491.117647058823
Catalogs : 4004.0
Medical : 612.0


We can see that Social Networking and Navigation apps have high numbers of installs; however, those are also categories that tend to be dominated by a small number of apps. Social networking apps require the building of a community--nobody is going to just download it and use it without some effort going into it. Since we would of course like to maximize our profit margins with less effort required, we will steer clear of that category.

Similarly, navigation apps require quite a lot of work, and we are not prepared to onboard the experts needed to make such an app a reality.

It does look like reference apps have a lot of reviews on average as well. Compared to social networking and navigation, this sounds a lot more digestable for our purposes. Given that the average number of reviews for a reference ap is just under 80,000, let's investigate apps with 100,000 + reviews, so we can get an idea of the upper echelon of these apps and see what we are 'competing' with.

In [152]:
for app in appstore_final:
    if app[11] == 'Reference' and int(app[5]) >= 100000:
        print(app)

['282935706', 'Bible', '92774400', 'USD', '0.0', '985920', '5320', '4.5', '5.0', '7.5.1', '4+', 'Reference', '37', '5', '45', '1']
['308750436', 'Dictionary.com Dictionary & Thesaurus', '111275008', 'USD', '0.0', '200047', '177', '4.0', '4.0', '7.1.3', '4+', 'Reference', '37', '0', '1', '1']


We can see that the Bible and the Dictionary.com app are drastically inflating our numbers here. The Bible has nearly 100 million reviews, and Dictionary.com has even more than that. Let's now look at reference apps with at least 10,000 reviews instead, to see if there could still be some value here.

In [156]:
for app in appstore_final:
    if app[11] == 'Reference' and int(app[5]) >= 10000:
        print(app)

['282935706', 'Bible', '92774400', 'USD', '0.0', '985920', '5320', '4.5', '5.0', '7.5.1', '4+', 'Reference', '37', '5', '45', '1']
['308750436', 'Dictionary.com Dictionary & Thesaurus', '111275008', 'USD', '0.0', '200047', '177', '4.0', '4.0', '7.1.3', '4+', 'Reference', '37', '0', '1', '1']
['364740856', 'Dictionary.com Dictionary & Thesaurus for iPad', '165748736', 'USD', '0.0', '54175', '10176', '4.5', '4.5', '4.0', '4+', 'Reference', '24', '5', '9', '1']
['414706506', 'Google Translate', '65281024', 'USD', '0.0', '26786', '27', '3.5', '4.5', '5.10.0', '4+', 'Reference', '37', '5', '59', '1']
['388389451', 'Muslim Pro: Ramadan 2017 Prayer Times, Azan, Quran', '100551680', 'USD', '0.0', '18418', '706', '4.5', '5.0', '9.2.1', '4+', 'Reference', '37', '5', '16', '1']
['1130829481', 'New Furniture Mods - Pocket Wiki & Game Tools for Minecraft PC Edition', '52959232', 'USD', '0.0', '17588', '17588', '4.5', '4.5', '1.0', '4+', 'Reference', '38', '3', '2', '1']
['399452287', 'Merriam-Webst

The short answer is maybe. It is certainly possible we could come up with a reference app that would do well for us, but given that so few apps dominate the reviews for this category, and given that we do not already have a product in production, we can liekly do better than this with less effort.

While not containing the highest number of ratings, there are a decent quantity of reviews for video and photo editors that are less well known. Let's take a look at those apps who have at least the 'average' number of ratings:

In [157]:
for app in appstore_final:
    if app[11] == 'Photo & Video' and int(app[5]) >= 28441:
        print(app)

['389801252', 'Instagram', '113954816', 'USD', '0.0', '2161558', '1289', '4.5', '4.0', '10.23', '12+', 'Photo & Video', '37', '0', '29', '1']
['447188370', 'Snapchat', '203038720', 'USD', '0.0', '323905', '576', '2.5', '3.0', '10.9.2.0', '12+', 'Photo & Video', '37', '0', '22', '1']
['544007664', 'YouTube - Watch Videos, Music, and Live Streams', '124673024', 'USD', '0.0', '278166', '0', '2.5', '0.0', '12.21', '17+', 'Photo & Video', '37', '4', '34', '1']
['448639966', 'Pic Collage - Picture Editor & Photo Collage Maker', '109210624', 'USD', '0.0', '123433', '1521', '5.0', '5.0', '7.12.17', '12+', 'Photo & Video', '37', '5', '12', '1']
['844570015', 'Funimate video editor: add cool effects to videos', '104902656', 'USD', '0.0', '123268', '64', '4.5', '4.5', '6.5.7', '12+', 'Photo & Video', '37', '4', '7', '1']
['835599320', 'musical.ly - your video social network', '173910016', 'USD', '0.0', '105429', '0', '4.5', '0.0', '5.7.3', '12+', 'Photo & Video', '37', '0', '21', '1']
['530957474

The biggest players in this category are common apps such as Instagram, Youtube, Snapchat and Google Photos, but there are a lot of lesser ubiqitous apps that still seem very popular. Taking photos is very popular in the age of smart phones and selfies, so this category seems like a reasonable approach. These types of apps are common, but not as saturarted like games and entertainment are, and they can provide a practical use. Before moving into development our app, we would want to find our "niche" in the photo and video editing space, but that is something we can investigate.

Before we get ahead of ourselves, though, let's also check the Google Play store. If we don't see similar results on the Play Store, we may want to rethink our approach--after all, we will be developing our app for Android devices first, so if it doesn't pass the 'Play Test' we won't even begin development on an iOS version.

In [141]:
display_table(playstore_final, 5) # the Installs columns

1,000,000+ : 0.1575497287522604
100,000+ : 0.11539330922242315
10,000,000+ : 0.10567359855334539
10,000+ : 0.10194394213381555
1,000+ : 0.0839737793851718
100+ : 0.06928119349005425
5,000,000+ : 0.06826401446654612
500,000+ : 0.05560578661844485
50,000+ : 0.04769439421338156
5,000+ : 0.04486889692585895
10+ : 0.035375226039783
500+ : 0.03243670886075949
50,000,000+ : 0.022830018083182642
100,000,000+ : 0.021360759493670885
50+ : 0.019213381555153706
5+ : 0.007911392405063292
1+ : 0.005085895117540687
500,000,000+ : 0.0027124773960217
1,000,000,000+ : 0.0022603978300180833
0+ : 0.0004520795660036166
0 : 0.00011301989150090416


The above information from the Google Play dataset shows us the number of installs, which is information we didn't have in the Apple Store dataset. However, we do not have very precise numbers. Even so, we don't really require that for our purposes. We will assume the low end of each installation category; for example, if an app has 5,000,000+ installs, we will assume 5,000,000; if an app has 1,000 + installs, we will assume 1,000.

In [158]:
genres_playstore = freq_table(playstore_final, 9)

for genre in genres_playstore:
    total = 0
    len_genre = 0
    for app in playstore_final:
        app_genre = app[9]
        if app_genre == genre:
            num_installs = app[5]
            num_installs = num_installs.replace(',', '')
            num_installs = num_installs.replace('+', '')
            total += float(num_installs)
            len_genre += 1
    avg_num_ratings = total / len_genre
    print(genre, ':', avg_num_ratings)
        

Art & Design : 2122850.9433962265
Art & Design;Creativity : 285000.0
Auto & Vehicles : 647317.8170731707
Beauty : 513151.88679245283
Books & Reference : 8814199.78835979
Business : 1712290.1474201474
Comics : 847380.1886792453
Comics;Creativity : 50000.0
Communication : 38590581.08741259
Dating : 854028.8303030303
Education : 550185.4430379746
Education;Creativity : 2875000.0
Education;Education : 4759517.0
Education;Pretend Play : 1800000.0
Education;Brain Games : 5333333.333333333
Entertainment : 5602792.775092937
Entertainment;Brain Games : 3314285.714285714
Entertainment;Creativity : 4000000.0
Entertainment;Music & Video : 6413333.333333333
Events : 253542.22222222222
Finance : 1387692.475609756
Food & Drink : 1924897.7363636363
Health & Fitness : 4188821.9853479853
House & Home : 1360598.042253521
Libraries & Demo : 638503.734939759
Lifestyle : 1421219.9096209912
Lifestyle;Pretend Play : 10000000.0
Card : 3815462.5
Arcade : 23028171.411042944
Puzzle : 8302861.91
Racing : 15910645.

There isn't a clear Photo & Video category here like in the Apple Store, but the 'Photography' category appears comparable. There are a high number of average installs, so let's look at what sorts of apps exist in this space.

In [162]:
for app in playstore_final:
    if app[9] == 'Photography':
        print(app)

['TouchNote: Cards & Gifts', 'PHOTOGRAPHY', '4.1', '19232', '28M', '1,000,000+', 'Free', '0', 'Everyone', 'Photography', 'August 6, 2018', '7.4.2', '4.4 and up']
['FreePrints – Free Photos Delivered', 'PHOTOGRAPHY', '4.8', '109500', '37M', '1,000,000+', 'Free', '0', 'Everyone', 'Photography', 'August 2, 2018', '2.18.2', '4.1 and up']
['Groovebook Photo Books & Gifts', 'PHOTOGRAPHY', '4.0', '21159', '36M', '500,000+', 'Free', '0', 'Everyone', 'Photography', 'January 31, 2018', '2.1.9', '4.0.3 and up']
['Moony Lab - Print Photos, Books & Magnets ™', 'PHOTOGRAPHY', '4.7', '1320', '9.5M', '50,000+', 'Free', '0', 'Everyone', 'Photography', 'June 27, 2018', '2.1.9', '4.1 and up']
['LALALAB prints your photos, photobooks and magnets', 'PHOTOGRAPHY', '4.7', '50424', '82M', '1,000,000+', 'Free', '0', 'Everyone', 'Photography', 'August 2, 2018', '584', '4.1 and up']
['Snapfish', 'PHOTOGRAPHY', '4.1', '32398', '28M', '1,000,000+', 'Free', '0', 'Everyone', 'Photography', 'August 1, 2018', '9.6.4',

Like with the Apple Store, we see a lot of photo editors with a large number of users, even outside of the usual suspects. Additional analysis would be required before proceeding, but this seems like a reasonable category to focus on for our app. 

Additional analysis that could help us in the development of our app includes:
    - Finding specifically what apps are the most popular, and of those which are easiest to emulate
    - Identifying what 'spin' we could put on a familiar format in order to stand out in the market